# LAB 4: Image Classification

## Step 1: Create project/folder and download data

Download [lab-4.zip](https://drive.google.com/file/d/1vjt-_R1YnpIkxoytykU_ZZ5CLNP7fVgi/view?usp=sharing) and unzip it. Place the `train/` and `test/` folders into `lab-4/` folder you create on your Google drive. These folders include images of five 'symbol' cards as seen from a small robot's camera. Also copy `lab4.ipynb` into the `lab-4/`.

**Name:** Saif Mustafa

**Student Number:** 1428093

## Step 2: Implement feature extraction

Below is a skeleton code for an image classification class called ImageClassifier as well as code for creating, training, and testing a classifier with the provided data sets. The three functions you will need to implement are indicated with comments in the code.

The first one of these is `extract_image_features` which should return a Numpy array that contains the features that represent the image. Before extracting any features, you should apply Gaussian blurring to your images to get rid of random sensor noice that is common in many lower cost cameras. For this, look into the filters module of scikit-image. Then explore at least two different types of features provided in the feature module of scikit-image. Inspect the size of the feature vectors generated by different methods and what the features look like for different images from the training or testing set. 

You will not yet get a good sense of how well each feature performs in allowing the classifiers to discriminate between different images. Hence, keep your code for extracting different features around until you explore classification performance in the next step. You can do that by duplicating the function with different names for different features.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import re

from sklearn import svm, metrics
from skimage import io, feature, filters, exposure, color, exposure

from sklearn.externals import joblib

from google.colab import drive
drive.mount('/content/drive')

class FeatureExtractor:
    
    def __init__(self):
        self.classifier = None
        self.folder = '/content/drive/MyDrive/Colab Notebooks/lab-4/lab4-data/'

    def imread_convert(self, f):
        return io.imread(f).astype(np.uint8)

    def save_classifier(self):
        joblib.dump(self.classifier, self.folder + 'classifier.joblib')

    def load_data_from_folder(self, dir):
        # read all images into an image collection
        ic = io.ImageCollection(self.folder + dir + '*.bmp',
                                load_func=self.imread_convert)

        # create one large array of image data
        data = io.concatenate_images(ic)
        
        # extract labels from image names
        labels = np.array(ic.files)
        for i, f in enumerate(labels):
            m = re.search('_', f)
            labels[i] = (f[len(dir):m.start()]).split('/')[-1]
        
        return(data,labels)
    
    def extract_image_features(self, data, feature):
        ########################################################################
        ############################ YOUR CODE HERE ############################
        ########################################################################

        vectors = [] # will store 1D vectorized images

        for img in data:
          img = filters.gaussian(img, multichannel=False) # apply gaussian
          gray_img = color.rgb2gray(img) # apply grayscale
          vectors.append(feature(gray_img).flatten()) # flatten -> makes it a 1D array

        return vectors


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Here is the code for creating an instance of the class, loading the data, and extracting the features.

In [ ]:
img_clf = FeatureExtractor()

# load images
print('Loading training set...')
(train_raw, train_labels) = img_clf.load_data_from_folder('train/')
print('Loading testing set...')
(test_raw, test_labels) = img_clf.load_data_from_folder('test/')
print()

# convert images into features, example using hog features
print('Extracting HOG features...')
train_data_hog = img_clf.extract_image_features(train_raw, feature.hog)
test_data_hog = img_clf.extract_image_features(test_raw, feature.hog)

# repeat with at least one other feature type...
print('Extracting Canny features...')
train_data_canny = img_clf.extract_image_features(train_raw, feature.canny)
test_data_canny = img_clf.extract_image_features(test_raw, feature.canny)

# repeat with at least one other feature type...
print('Extracting Peak Local Max features...')
train_data_plm = img_clf.extract_image_features(train_raw, feature.peak_local_max)
test_data_plm = img_clf.extract_image_features(test_raw, feature.peak_local_max)

Loading training set...
Loading testing set...

Extracting HOG features...
Extracting Canny features...
Extracting Peak Local Max features...


In [ ]:
# Inspecting the features.
print('HOG Feature Vector')
print(train_data_hog[0])
print(f'Vector Size: {len(train_data_hog[0])}') # Vector Size: 86184
print() 

print('##########################################################################')
print()

# repeat with at least one other feature type...

# Inspecting the features.
print('Canny Feature Vector')
print(train_data_canny[0])
print(f'Vector Size: {len(train_data_canny[0])}') # Vector Size: 720
print()

print('##########################################################################')
print()

# Inspecting the features.
print('Peak Local Max Vector')
print(train_data_plm[0])
print(f'Vector Size: {len(train_data_plm[0])}')
print()

print('##########################################################################')
print()

HOG Feature Vector
[0.04775442 0.03536229 0.03683505 ... 0.01790003 0.04655878 0.04953976]
Vector Size: 86184

##########################################################################

Canny Feature Vector
[False False False ... False False False]
Vector Size: 76800

##########################################################################

Peak Local Max Vector
[237  35 236 ...  78   1  77]
Vector Size: 2866

##########################################################################



## Step 3: Implement classifier training and testing
Next, implement the `train_classifier` and `predict_labels` functions for at least two different types of classifiers. After this, the second cell below shoul then produce performance results of the classifier.

Explore the performance of at least **two feature types** and at least **two classifiers** (i.e. at least four different combinations) in terms of classification **performance** on the test set. Update the code to display the **F1 score** on the test set for the **four different combinations** with informative prompts. Then the code should display the **detailed performance** (confusion matrix, accuracy, F1 score) that is already displayed only for the best performing combination of features and classifier. Also make sure the best performing classifier is saved onto your drive for the next step of the lab.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import sklearn.metrics as skmetrics
from sklearn.model_selection import cross_val_score 

# Continue implementation of the ImageClassifier class in this cell
class ImageClassifier(FeatureExtractor):

    def train_classifier(self, train_data, train_labels, classifier=KNeighborsClassifier):
        
        self.classifier = classifier()
        self.classifier.fit(train_data, train_labels)

    def predict_labels(self, data):
        
        predicted_labels = self.classifier.predict(data)
        
        return predicted_labels       

In [ ]:
print(test_labels)

['drone' 'drone' 'drone' 'drone' 'drone' 'hands' 'hands' 'hands' 'hands'
 'hands' 'inspection' 'inspection' 'inspection' 'inspection' 'inspection'
 'none' 'none' 'none' 'none' 'none' 'order' 'order' 'order' 'order'
 'order' 'place' 'place' 'place' 'place' 'place' 'plane' 'plane' 'plane'
 'plane' 'plane' 'truck' 'truck' 'truck' 'truck' 'truck']


Re-run initialization and feature extraction, then train/test the classifier.

In [ ]:
from skimage.feature import hog, canny, peak_local_max
# https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation

img_clf = ImageClassifier()


# create list of list containing features you want to run through along with any
# associated metadata

# list of features to test, peak_local_max giving too many errors
features = [hog,canny]

# list of classifiers
classifiers = [KNeighborsClassifier, MLPClassifier]

# Variable to hold the best F1 score
best_f1 = 0

# Loop through the different combinantions of features and classifiers
for f in features:

  # train and test data before entering classifier loop
  train_data = img_clf.extract_image_features(train_raw, f)
  test_data = img_clf.extract_image_features(test_raw, f)

  for c in classifiers:  

    # Train model
    train = img_clf.train_classifier(train_data, train_labels, c)

    # Test model
    predicted_labels = img_clf.predict_labels(test_data)

    # Create confusion matrix
    confusion_matrix = skmetrics.confusion_matrix(test_labels, predicted_labels)
    print("Classifer:", c)
    print("Feature: ", f)
    print("Confusion Matrix:") 
    print(confusion_matrix)

    # Print test accuracy
    print("Accuracy:", skmetrics.accuracy_score(test_labels, predicted_labels))

    # Print test F1 score
    print("F1 Score:", skmetrics.f1_score(test_labels, predicted_labels, average='weighted'))

    print("")
    print("-----")
    print("")
    
    # Check if last model is better than the current best one and save it
    if(skmetrics.f1_score(test_labels, predicted_labels, average='weighted') > best_f1):
      best_f1 = skmetrics.f1_score(test_labels, predicted_labels, average='weighted')
      img_clf.save_classifier()

Classifer: <class 'sklearn.neighbors._classification.KNeighborsClassifier'>
Feature:  <function hog at 0x7f1ed3ab09e0>
Confusion Matrix:
[[4 0 0 0 0 0 0 1]
 [0 4 1 0 0 0 0 0]
 [0 0 5 0 0 0 0 0]
 [0 0 0 5 0 0 0 0]
 [0 1 0 0 2 2 0 0]
 [0 0 0 0 0 2 3 0]
 [0 0 0 1 0 0 4 0]
 [1 0 0 0 0 1 0 3]]
Accuracy: 0.725
F1 Score: 0.7153679653679654

-----

Classifer: <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>
Feature:  <function hog at 0x7f1ed3ab09e0>
Confusion Matrix:
[[1 0 0 0 4 0 0 0]
 [0 5 0 0 0 0 0 0]
 [0 1 4 0 0 0 0 0]
 [0 0 0 5 0 0 0 0]
 [0 0 0 0 5 0 0 0]
 [0 0 0 0 0 5 0 0]
 [0 0 0 1 0 0 4 0]
 [0 0 0 0 1 1 0 3]]
Accuracy: 0.8
F1 Score: 0.781881313131313

-----

Classifer: <class 'sklearn.neighbors._classification.KNeighborsClassifier'>
Feature:  <function canny at 0x7f1ed8c33f80>
Confusion Matrix:
[[0 0 0 5 0 0 0 0]
 [0 0 0 5 0 0 0 0]
 [0 0 0 5 0 0 0 0]
 [0 0 0 5 0 0 0 0]
 [0 0 0 5 0 0 0 0]
 [0 0 0 5 0 0 0 0]
 [0 0 0 5 0 0 0 0]
 [0 0 0 5 0 0 0 0]]
Accuracy: 0.125
F1 S

In [ ]:
print("Train data: {}".format(train_raw.shape))
print("Test data: {}".format(test_raw.shape))
print("Train labels: {}".format(train_labels.shape))
print("Test labels: {}".format(test_labels.shape))

In [ ]:
# TODO: Save the best model using the `save_classifier` method of the `FeatureExtractor` class.
# done in previous loop, ignore:
# img_clf.save_classifier()

## Step 4: Transfer classifier to your camera

Next you will apply your trained classifier directly onto images captured by your webcam. Since this is our last lab using Python we would like to give you the opportunity to install Python and Jupyter notebooks on your machine, and do this part of the lab as a conventional Python script. However, if you would rather not do that at the moment, you still have the option of doing this part with Colab Notebooks.

* **Option 1:** If you would like to do this part locally, first follow the Python Installation guidelines. Then download the skeleton camera capture script `camera.py` and test it out. Then update this script as described below and submit this script. Make sure you note in lab4.ipynb that you chose this option.
* **Option 2:** If you would like to do this part in Colab Notebooks, add your code below. Check out [`camera.ipynb`](https://colab.research.google.com/drive/1IfHqK83dDVyxsQzQwsnxc4CUrRUoL9y8) for sample code for capturing camera images in Colab notebooks.

Your code should first load the classifier you saved in Step 3 with the following line (already implemented in the sample code):

`classifier = joblib.load('classifier.joblib')`

The script should then go into a loop where it (1) captures a new image from the camera, (2) processes the image to make it grayscale and filter the noise, (3) extracts the features like you did in Step 2 using the right set of features for your trained classifier, (4) detects whether the image contains one of the seven images using the trained classifier, and (5) displays the detected class name on the image in every iteration. You can use the images printed on paper provided by the TAs to test your script with your camera.

In [ ]:
from IPython.display import HTML, Audio
from IPython.display import clear_output
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import io as io2
from PIL import Image
import time
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import cv2
from skimage.feature import hog

from skimage.color import rgb2gray

############################
####### CAMERA CODE ########
############################

VIDEO_HTML = """
<video autoplay
 width=%d height=%d style='cursor: pointer;'></video>
<script>

var video = document.querySelector('video')

navigator.mediaDevices.getUserMedia({ video: true })
  .then(stream=> video.srcObject = stream)

function getFrame() {
    var canvas = document.createElement('canvas')
    var [w,h] = [video.offsetWidth, video.offsetHeight]
    canvas.width = w
    canvas.height = h
    canvas.getContext('2d')
          .drawImage(video, 0, 0, w, h)
    return canvas.toDataURL('image/jpeg', 0.8)
}

</script>
"""

# Make sure the pictures taken by your camera match the size of the 
# training set – 320px x 240px
def start_camera(filename='photo.jpg', quality=0.8, size=(320,240)):
  display(HTML(VIDEO_HTML % (size[0],size[1])))

def take_photo(filename='photo.jpg', quality=0.8, size=(320,240)):
  data = eval_js('getFrame()')
  binary = b64decode(data.split(',')[1])
  f = io2.BytesIO(binary)
  return np.asarray(Image.open(f))

############################
############################
############################

class ImageClassifer_webcam(ImageClassifier):
  
  def load_classifier(self):
    print('Loading classifier...')
    self.classifier = joblib.load(self.folder + 'classifier.joblib')

# Create a new instance of the classifier
img_clf = ImageClassifer_webcam()

# Load the previously saved model
img_clf.load_classifier()

print("Classifier =", img_clf.classifier)

# Start camera and wait for it to "warm up"
start_camera()
time.sleep(3)

while(True):
  img = take_photo() # click

  ##############################################################################
  ############################ YOUR CODE HERE ##################################
  ##############################################################################

  # Convert to grayscale
  g_img = color.rgb2gray(img)

  # Extract features from the image just caputred
  extracted = hog(g_img)
  
  # Show frame and prediction
  pred = img_clf.predict_labels(extracted.reshape(1,-1))
  print("Prediction:",pred)

  time.sleep(1)


Loading classifier...
Classifier = MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)


Prediction: ['truck']
Prediction: ['order']


KeyboardInterrupt: ignored

## (Optional) Improve classifier

You will notice that your classifier is prone to errors when tested with your camera. Part of the reason is that the images were collected from a different camera. You can try improving the performance of your camera image classifier by re-training your classifier with images collected from your camera.

## Step 5: Submit your code on Canvas

Complete this lab by submitting a link to your updated Colab Notebook (and if you chose Option 1 in Step 4, by uploading your updated `camera.py`) on Canvas, by Oct 22 Tuesday, 11:59pm. We will test your code by running it and inspecting the classification results to make sure: 
* A comparison of  at least four combinations of feature types and classifiers were made
* A reasonable classification performance was achieved with the best combination (higher than random chance).

We will test your camera image classification code by running it and showing it the seven different printed images to check that more than half of the images can be recognized correctly in some configuration relative to the camera.

See Canvas for a grading rubric.

# Scratchpad (Please ignore)

In [ ]:
print(img.reshape(1,-1).shape)

In [ ]:
img = '/content/drive/MyDrive/Colab Notebooks/lab-4/lab4-data/train/drone_10T105759665802.bmp'
img = plt.imread(img)
g_img = color.rgb2gray(img)

  # Extract features from the image just caputred
extracted = feature.hog(g_img)
gaussian = filters.gaussian(extracted).flatten()

# Show frame and prediction
pred = img_clf.classifier.predict(gaussian.reshape(1,-1))
print("Prediction:",pred)
plt.imshow(img, cmap ='gray')
plt.show()

ValueError: ignored